In [14]:
%matplotlib  inline
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

# Notebook til beregning af transmissiviteter fra Jupiter på baggrund af renpumpningsdata
Notebooken anvende Cooper-Jacob metoden til itterativt at bestemme Transmissiviteten på baggrung af renpumpningsdata. Resultaterne vil være behæftet med stor usikkerhed, da transmissivitetsestimaterne ofte er påvirket af et filtertab. Sænkningerne er således ofte højere i boringen end det omkringliggende magasin.

Beregningerne er baseret på data fra [Jupiter Databasen](https://data.geus.dk/geusmap/?mapname=jupiter#baslay=baseMapDa&optlay=&extent=528681.642558052,6129998.970201795,624005.2428222394,6175247.5146309985&layers=jupiter_boringer_ws&filter_0=kode%3DV%26dgunr%3D%26anlaegsnavn.part%3D%26hovedtype%3D%26kommunenavn.part%3D%26dybde.min%3D%26dybde.max%3D%26aar.min%3D%26aar.max%3D%26formaal%3D%26anvendelse%3D%26dataejer%3D)

- For at finde relevante data skal vi starte med at filtrere boringerne, så der kun vises boringer hvis formål er vandforsyning på hjemmesiden
- Efterfølgende kan vi finde frem til de enkelte boringer vi vil se på. Hvis der er foretaget et pumpeforsøg i boringen vil det være opgjort under "Grundvand" - "Pumpninger". Se f.eks. her: [DGU 127.389](https://data.geus.dk/JupiterWWW/borerapport.jsp?borid=470457)

![Jupiter data](img/pumpedata.png)

Find nogle forskellige boringer i Jupiter databasen:
- Beregn transmissiviteten for 4-5 forskellige boringer. Husk at gemme de resultater i kommer frem til sammen med DGU nummeret, da i skal burge dem igen i en senere opgave
- Hvor følsom er beregningen over for det valgte magasintal?
- Hvad tror i der skal til for at få så pålidelige data som muligt fra prøvepumpningsanalysen?
- Hvis der er manglende data for nogle af boringerne (f.eks. varighed) Hvad betyder det så for resultatet hvis man "gætter" på en varighed af forsøget?

## Boringer med pumpedata:
![Boringer med pumpedata](img/boringer.png)

In [15]:
# funktion til itterativt at bestemme Transmissiviteten ved anvendelse af Cooper Jacob metoden.
def Beregn_T(Ydelse, Pumpetid, Saenkning, boringsdiameter, Smag= None):
    e = 1e-8 # stopkriterie for itterationer over transmissiviteter
    
    
    if Smag is None:
        Smag = 0.002  # defaultværdi for Magasintallet
    
    if Saenkning <= 0.:
        T = -1  # hvis sænkningen er 0 sættes transmissiviteten til -1
    else:
        T = 0.0001
        Tini = 0.001
        
        fac = (2.3/(4*np.pi))*(Ydelse/Saenkning)
    
        c = 0
        while (np.abs(Tini-T) > e) or c==30:
            Tini = T
            T = fac * np.log10((2.25*Tini*Pumpetid)/(boringsdiameter**2*Smag))
            c+=1          
            
    return T # returner den estimerede transmissivitet

I den efterfølgende celle beregnes transmissivitetn på baggrund af data. Der regnes for et sæt af magasintal således at følsomheden kan bestemmes. I de videre beregninger er den maksimale transmissivitet anvendt

In [17]:
Ydelse = 50.5  # ydelse i m3/time
Pumpetid =  90.  # Pumpetid i timer
Saenkning = 14.87  # Sænkning i meter
boringsdiameter = 225/1000.  # diameter på filteret der pumpes fra i meter
Magasintal = 0.0001

In [19]:
Beregn_T(Ydelse, Pumpetid, Saenkning, boringsdiameter, Smag= None)/(60*60)

0.0011975144099925898